In [2]:
import pandas as pd
import numpy as np 
import json
import csv


import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook as tqdm

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('averaged_perceptron_tagger')

import spacy
import math

import string
import sys
import random

from collections import Counter
from itertools import chain

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aarunku5/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
nlp = spacy.load("en_trf_bertbaseuncased_lg")



In [12]:
testgood=pd.read_csv("/Users/aarunku5/Desktop/DQI/DQI_Worker/data/SNLI/AFLITE_SPLIT/Final/Test/P7/testgood.csv")
traingood=pd.read_csv("/Users/aarunku5/Desktop/DQI/DQI_Worker/data/SNLI/AFLITE_SPLIT/Final/Test/P7/traingood_reduced.csv")

In [7]:
testgoodsen=pd.read_csv("./test_good_101_1.csv")
testgoodsen


,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,"['neutral', 'neutral', 'neutral', 'neutral', '...",317383917.jpg#3,neutral,317383917.jpg#3r1n,A group of dogs stand in the snow.,( ( ( A group ) ( of dogs ) ) ( ( stand ( in (...,(ROOT (S (NP (NP (DT A) (NN group)) (PP (IN of...,Some animals are forced to stand still in snow.,( ( Some animals ) ( ( are ( forced ( to ( ( s...,(ROOT (S (NP (DT Some) (NNS animals)) (VP (VBP...
1,"['entailment', 'neutral', 'neutral', 'neutral'...",5821129557.jpg#3,neutral,5821129557.jpg#3r1e,A little boy is getting is birthday cake and i...,( ( ( A ( little boy ) ) ( is getting ) ) ( ( ...,(ROOT (S (NP (NP (DT A) (JJ little) (NN boy)) ...,A child at a party.,( ( ( A child ) ( at ( a party ) ) ) . ),(ROOT (NP (NP (DT A) (NN child)) (PP (IN at) (...
2,"['neutral', 'neutral', 'neutral', 'neutral', '...",510791586.jpg#3,neutral,510791586.jpg#3r1n,A lit girl splashes around in natural water.,( ( A ( lit girl ) ) ( ( ( splashes around ) (...,(ROOT (S (NP (DT A) (NN lit) (NN girl)) (VP (V...,The girl is outside.,( ( The girl ) ( ( is outside ) . ) ),(ROOT (S (NP (DT The) (NN girl)) (VP (VBZ is) ...
3,"['neutral', 'neutral', 'neutral', 'entailment'...",2373018323.jpg#1,neutral,2373018323.jpg#1r1n,A young man in a blue hoodie doing a flip off ...,( ( ( ( A ( young man ) ) ( in ( ( a ( blue ho...,(ROOT (NP (NP (NP (DT A) (JJ young) (NN man)) ...,A young man does a flip off a wall and wares b...,( ( A ( young man ) ) ( ( ( does ( a flip ) ) ...,(ROOT (S (NP (DT A) (JJ young) (NN man)) (VP (...
4,"['neutral', 'contradiction', 'neutral', 'neutr...",4799796561.jpg#4,neutral,4799796561.jpg#4r1n,A man with a cigarette sits on the stoop.,( ( ( A man ) ( with ( a cigarette ) ) ) ( ( s...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN with...,An old man is smoking a cigar.,( ( An ( old man ) ) ( ( is ( smoking ( a ciga...,(ROOT (S (NP (DT An) (JJ old) (NN man)) (VP (V...
...,...,...,...,...,...,...,...,...,...,...
96,"['entailment', 'contradiction', 'entailment', ...",4787038693.jpg#2,entailment,4787038693.jpg#2r1e,"A man sitting on a statue outdoors, near a wom...",( ( ( A man ) ( ( ( ( sitting ( on ( a statue ...,(ROOT (NP (NP (DT A) (NN man)) (VP (VBG sittin...,man and woman sitting near a statue,( ( man ( and ( woman sitting ) ) ) ( near ( a...,(ROOT (NP (NP (NN man) (CC and) (NN woman) (NN...
97,"['neutral', 'neutral', 'entailment', 'neutral'...",6073765731.jpg#2,neutral,6073765731.jpg#2r1n,A lady is helping another woman work in a silv...,( ( A lady ) ( ( is ( ( helping ( another ( wo...,(ROOT (S (NP (DT A) (NN lady)) (VP (VBZ is) (V...,Two woman are trying to finish orders from a d...,( ( Two woman ) ( are ( trying ( to ( ( finish...,(ROOT (S (NP (CD Two) (NN woman)) (VP (VBP are...
98,"['neutral', 'neutral', 'entailment', 'neutral'...",3801367099.jpg#2,neutral,3801367099.jpg#2r1n,A man in a leaf-patterned short stand behind a...,( ( ( A man ) ( in ( ( a ( leaf-patterned ( sh...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN in) ...,The man is outdoors.,( ( The man ) ( ( is outdoors ) . ) ),(ROOT (S (NP (DT The) (NN man)) (VP (VBZ is) (...
99,"['neutral', 'contradiction', 'neutral', 'contr...",212863746.jpg#2,contradiction,212863746.jpg#2r1n,Two women are getting their pictures taken wit...,( ( Two women ) ( ( are ( getting ( ( their pi...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,The women are in their underwear.,( ( The women ) ( ( are ( in ( their underwear...,(ROOT (S (NP (DT The) (NNS women)) (VP (VBP ar...


In [13]:
s=testgoodsen.iloc[100]['sentence1']+' '+testgoodsen.iloc[100]['sentence2']
s=s.replace(",","")
s=s.replace(".","")
s=s.replace("'","")
s=s.replace("]","")
s=s.replace("[","")
s=s.replace("\"","")    
s

'A woman in a green shirt preparing to run on a treadmill A woman is preparing to sleep on a treadmill'

In [14]:
arr1=[]
for i in tqdm(range(len(traingood))):
    ss=traingood.iloc[i]['sentence1']+' '+str(traingood.iloc[i]['sentence2'])
    ss=ss.replace(",","")
    ss=ss.replace(".","")
    ss=ss.replace("'","")
    ss=ss.replace("]","")
    ss=ss.replace("[","")
    ss=ss.replace("\"","")    
    arr1.append(ss)

In [20]:
arr=[]
for i in tqdm(range(len(arr1))):
        arr.append(nlp(s).similarity(nlp(arr1[i])))


In [21]:
testgood['100']=arr
testgood

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.071882,0.025165,0.018092,0.052241,0.069777,0.116153,0.022933,0.049636,0.029415,0.150562,...,0.038231,0.214497,0.039641,0.019544,0.062021,0.054531,0.060290,0.063684,0.033047,0.683835
1,0.034329,0.030757,0.023462,0.079779,0.072158,0.133115,0.023238,0.031615,0.035262,0.210235,...,0.050891,0.323096,0.055131,0.028565,0.050924,0.058543,0.052305,0.061536,0.039537,0.701413
2,0.084056,0.040216,0.031271,0.081800,0.074059,0.194731,0.029552,0.047705,0.042459,0.170558,...,0.054026,0.256349,0.083892,0.041539,0.082822,0.065571,0.060080,0.089516,0.050475,0.678789
3,0.036970,0.023408,0.013202,0.061989,0.059313,0.099428,0.015309,0.023490,0.024565,0.114242,...,0.034381,0.231165,0.047663,0.020428,0.052764,0.071112,0.031168,0.040751,0.030133,0.591433
4,0.055443,0.047610,0.042488,0.145855,0.122316,0.256886,0.107199,0.055443,0.095104,0.301995,...,0.068771,0.252556,0.094771,0.102499,0.124382,0.126559,0.060466,0.154399,0.061922,0.734385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91594,0.088297,0.128787,0.073362,0.254783,0.267090,0.143741,0.144553,0.094531,0.101206,0.153472,...,0.080119,0.188305,0.139580,0.067165,0.257427,0.231726,0.107770,0.222014,0.117212,0.776085
91595,0.082187,0.167663,0.155793,0.279392,0.221880,0.109569,0.152219,0.117290,0.173881,0.045141,...,0.167332,0.079400,0.203119,0.059985,0.323879,0.179236,0.168579,0.287138,0.169869,0.726033
91596,0.160580,0.263491,0.082639,0.158260,0.059272,0.062429,0.066519,0.072063,0.241643,0.057413,...,0.117713,0.044812,0.173057,0.054398,0.169914,0.434260,0.192508,0.154696,0.174402,0.791925
91597,0.170418,0.145652,0.159169,0.155263,0.106684,0.095047,0.217085,0.114639,0.184096,0.110625,...,0.137819,0.056908,0.116003,0.144780,0.258367,0.146981,0.261515,0.134989,0.095186,0.747161


In [22]:
maxtest=testgood.max(axis=1)
x=[0.2,0.3,0.4]
for i in range(3):
    print(1/((abs(maxtest-x[i])).sum()/100))

0.0018947533552262866
0.002292662083666188
0.0029021243071519773
